<a href="https://colab.research.google.com/github/Biniyamgd/IDP/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dlib
!pip install face_recognition
from google.colab import drive
drive.mount('/content/drive')
import firebase_admin
from firebase_admin import credentials
import os

if not os.path.exists('./unknown'):
   os.mkdir('./unknown')


cred = credentials.Certificate("/content/drive/MyDrive/serviceAccountKey.json")
firebase_admin.initialize_app(cred,{
    "databaseURL":"https://facerecondb-default-rtdb.firebaseio.com/",
    "storageBucket":"facerecondb.appspot.com"
})


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=29d28da42c236d46e21cd766e4fb0b7daa707c0a47d0df6e0a54b694130149a4
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models
Mounted at /content/drive


In [2]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='./unknown/photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');



      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);

      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      capture.style.width = '160px';
      capture.style.height = '30px';
      capture.style.border="none";
      capture.style.color="black";
      capture.style.borderRadius="7px";
      capture.style.backgroundColor="rgb(100,165,145)";
      div.appendChild(capture);

      video.srcObject = stream;
      await video.play();
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image, clear_output
from google.colab.patches import cv2_imshow

import cv2
import os
import face_recognition
import numpy as np
import firebase_admin
from firebase_admin import storage
from firebase_admin import db
import time
from datetime import datetime

def Retake():
 try:
   filename = take_photo()
 except Exception as err:
   print(str(err))



 filepath='./drive/MyDrive/students'
 folder=os.listdir(filepath)
 modelist=[]
 studentId=[]


 for Path in folder:
  if os.path.isfile(os.path.join(filepath, Path)) and Path.lower().endswith(('.png', '.jpg', '.jpeg')):
    modelist.append(cv2.imread(os.path.join(filepath,Path)))
    studentId.append(os.path.splitext(Path)[0])
    imgfile=os.path.join(filepath,Path)
    relative_path = os.path.relpath(imgfile, filepath)
    bucket=storage.bucket()
    blob = bucket.blob(relative_path)
    blob.upload_from_filename(imgfile)


 encodeList=[]
 def encodingImg(imgList):
    for img in imgList:
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        encode=face_recognition.face_encodings(img)
        encodeList.append(encode)
    return encodeList


 encodes=encodingImg(modelist)

 filepath2=cv2.imread(os.path.join('/content/unknown','photo.jpg'))

 def encodingImg2(img):
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        encode=face_recognition.face_encodings(img)
        return encode

 encodehello = encodingImg2(filepath2)
 i=0
 Matches=[]
 FaceDistance=[]

 try:
  while i<len(encodes):
    for items in encodes[i]:
      match1=face_recognition.compare_faces(items,encodehello)
      facedis=face_recognition.face_distance(items,encodehello)
      Matches.append(match1[0])
      FaceDistance.append(facedis[0])
    i=i+1
 except Exception as err:
     print("Retake The Photo Please")
     Retake()
     return
 if True in Matches:
   checked=cv2.imread('/content/drive/MyDrive/display/checked.jpg')
   cv2_imshow(checked)
   index=np.argmin(FaceDistance)
   ID=studentId[index]
   IDref=db.reference(f'students/{ID}')
   studentInfo=IDref.get()
   datetimeObject = datetime.strptime(studentInfo['last_attendance_time'],"%Y-%m-%d %H:%M:%S")
   secondsElapsed = (datetime.now() - datetimeObject).total_seconds()
   if secondsElapsed > 60:
     studentInfo['Total_attendance'] += 1
     IDref.child('Total_attendance').set(studentInfo['Total_attendance'])
     IDref.child('last_attendance_time').set(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
     clear_output()


     Downblob=bucket.get_blob(f'{ID}.jpg')
     array=np.frombuffer(Downblob.download_as_string(),np.uint8)
     image1=cv2.imdecode(array,cv2.IMREAD_COLOR)

     img2 = cv2.imread('/content/drive/MyDrive/display/info.jpg')
     text = f"{studentInfo['name']}"

     img1 = cv2.resize(image1, (300, 400))
     img2 = cv2.resize(img2, (300, 400))

     position = (75, 105)
     font = cv2.FONT_HERSHEY_SIMPLEX
     font_scale = 0.7
     color = (255, 255, 255)
     thickness = 2
     cv2.putText(img2, text, position, font, font_scale, color, thickness)

     text = f"ETS{ID}/13"
     position = (45, 170)
     font = cv2.FONT_HERSHEY_SIMPLEX
     font_scale = 0.7
     color = (255, 255, 255)
     thickness = 2
     cv2.putText(img2, text, position, font, font_scale, color, thickness)

     text = f"{studentInfo['department']}"
     position = (137, 235)
     font = cv2.FONT_HERSHEY_SIMPLEX
     font_scale = 0.7
     color = (255, 255, 255)
     thickness = 2

     cv2.putText(img2, text, position, font, font_scale, color, thickness)

     text = f"{studentInfo['course']}"
     position = (88, 302)
     font = cv2.FONT_HERSHEY_SIMPLEX
     font_scale = 0.7
     color = (255, 255, 255)
     thickness = 2
     cv2.putText(img2, text, position, font, font_scale, color, thickness)

     text = f"{studentInfo['Total_attendance']}"
     position = (185, 360)
     font = cv2.FONT_HERSHEY_SIMPLEX
     font_scale = 0.7
     color = (255, 255, 255)
     thickness = 2
     cv2.putText(img2, text, position, font, font_scale, color, thickness)


     img_combined = cv2.hconcat([img1, img2])
     cv2_imshow(img_combined)
     delay=10
     time.sleep(delay)
     clear_output()
     Retake()
   else:
      clear_output()
      al_img=cv2.imread('/content/drive/MyDrive/display/already.jpg')
      cv2_imshow(al_img)
      delay=5
      time.sleep(delay)
      clear_output()
      Retake()
 else:
    un_img=cv2.imread('/content/drive/MyDrive/display/unregistered.jpg')
    cv2_imshow(un_img)
    delay=5
    time.sleep(delay)
    clear_output()
    Retake()
Retake()

<IPython.core.display.Javascript object>